TODO:
    
- Reduce the time to caculate the boundary (keep the last one add the new one)
- Look for the c's within the boundary of the current region


In [1]:
from z3 import *
from utils import *

In [2]:
from sklearn.datasets import load_breast_cancer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.tree import _tree, plot_tree
import pandas as pd
from matplotlib import pyplot as plt
import pickle
import time
import copy
import tqdm
from pprint import pprint

In [3]:
def create_adv_sample_x(x_adv_dict, sample):
    """
    Finds an adversarial sample that is close to real sample. 
    
    Args:
        x_adv_dict: a dictionary showing the boundaries of the search region
            the keys are the the var_x names, x(i_v), and the values are
            set of True, False, or empty. 
        sample: the sample used to find the adversarial example for. 
    Returns:
        array-like: 
            the adversarial example
        dataframe: 
            a dataframe comparing the sample and adversarial example
    """
    x_adv = [list(v)[0] for k,v in x_adv_dict.items() if len(v)==1]
    columns = [k[2:-1] for k,v in x_adv_dict.items() if len(v)==1]    
    x_adv = pd.DataFrame(data=[x_adv], columns=columns)
        
    num_features = sample.shape[1]
    x_adv_num = np.zeros((1, num_features))
    
    bound_i = []

    for i in range(num_features):
        col_i = np.asarray([ci for ci in columns if int(ci.split("_")[0]) == i])
        col_i_false = col_i[np.where(x_adv[col_i].values == False)[1]]
        col_i_true = col_i[np.where(x_adv[col_i].values == True)[1]]

        if len(col_i_false) > 0 and len(col_i_true) > 0:
            max_i = np.asarray([float(ci.split("_")[1]) for ci in col_i_false])
            max_i = max_i.min()
            min_i = np.asarray([float(ci.split("_")[1]) for ci in col_i_true])
            min_i = min_i.max()

            if sample[0, i] <= min_i:
                x_adv_num[0, i] = min_i + (max_i - min_i) * 0.001
            elif sample[0, i] >= max_i:
                x_adv_num[0, i] = max_i - (max_i - min_i) * 0.001
            else:
                x_adv_num[0, i] = sample[0, i]

        elif len(col_i_false) > 0 and len(col_i_true) == 0:
            min_i = -np.inf
            max_i = np.asarray([float(ci.split("_")[1]) for ci in col_i_false])
            max_i = max_i.min()
            
            if sample[0, i] <= max_i:
                x_adv_num[0, i] = sample[0, i]
            else:
                x_adv_num[0, i] = max_i * (1 - np.sign(max_i) * 0.001)
                

        elif len(col_i_false) == 0 and len(col_i_true) > 0:
            min_i = np.asarray([float(ci.split("_")[1]) for ci in col_i_true])
            min_i = min_i.max()
            max_i = np.inf
            
            if sample[0, i] >= min_i:
                x_adv_num[0, i] = sample[0, i]
            else:
                x_adv_num[0, i] = min_i * (1 + np.sign(min_i) * 0.001)
            

        else:
            min_i = -np.inf
            max_i = np.inf
            x_adv_num[0, i] = sample[0, i]

        bound_i.append([min_i, max_i])

    data_comp = np.r_[sample, x_adv_num]
    compare = pd.DataFrame(
        data=data_comp, columns=[str(i) for i in range(data_comp.shape[1])]
    ).T
    compare["diff(%)"] = np.abs(compare[1] - compare[0]) / np.abs(compare[0])
    compare["bound"] = bound_i

    return x_adv_num, compare

In [4]:
def create_adv_sample(x_adv_dict, sample):
    """
    Finds a random adversarial sample with in the bounds given by `x_adv_dict`. 
    
    Args:
        x_adv_dict: a dictionary showing the boundaries of the search region
            the keys are the the var_x names, x(i_v), and the values are
            set of True, False, or empty. 
        sample: the sample used to find the adversarial example for. 
    Returns:
        array-like: 
            the adversarial example
        dataframe: 
            a dataframe comparing the sample and adversarial example
    """
    x_adv = [list(v)[0] for k,v in x_adv_dict.items() if len(v)==1]
    columns = [k[2:-1] for k,v in x_adv_dict.items() if len(v)==1]    
    x_adv = pd.DataFrame(data=[x_adv], columns=columns)
        
    num_features = sample.shape[1]
    x_adv_num = np.zeros((1, num_features))
    
    bound_i = []

    for i in range(num_features):
        col_i = np.asarray([ci for ci in columns if int(ci.split("_")[0]) == i])
        col_i_false = col_i[np.where(x_adv[col_i].values == False)[1]]
        col_i_true = col_i[np.where(x_adv[col_i].values == True)[1]]

        if len(col_i_false) > 0 and len(col_i_true) > 0:
            max_i = np.asarray([float(ci.split("_")[1]) for ci in col_i_false])
            max_i = max_i.min()
            min_i = np.asarray([float(ci.split("_")[1]) for ci in col_i_true])
            min_i = min_i.max()

            if sample[0, i] <= min_i:
                x_adv_num[0, i] = min_i + (max_i - min_i) * 0.001
            elif sample[0, i] >= max_i:
                x_adv_num[0, i] = max_i - (max_i - min_i) * 0.001
            else:
                x_adv_num[0, i] = sample[0, i]

        elif len(col_i_false) > 0 and len(col_i_true) == 0:
            min_i = -np.inf
            max_i = np.asarray([float(ci.split("_")[1]) for ci in col_i_false])
            max_i = max_i.min()
            
            x_adv_num[0, i] = np.random.uniform(-100 * np.abs(max_i * (1 - np.sign(max_i) * 0.001)), max_i * (1 - np.sign(max_i) * 0.001))

        elif len(col_i_false) == 0 and len(col_i_true) > 0:
            min_i = np.asarray([float(ci.split("_")[1]) for ci in col_i_true])
            min_i = min_i.max()
            max_i = np.inf
                      
            x_adv_num[0, i] = np.random.uniform(min_i * (1 + np.sign(min_i) * 0.001), np.abs(min_i * (1 + np.sign(min_i) * 0.001)*100))

        else:
            min_i = -np.inf
            max_i = np.inf
            x_adv_num[0, i] = np.random.uniform(-10000, 10000) # sample[0, i]

        bound_i.append([min_i, max_i])

    data_comp = np.r_[sample, x_adv_num]
    compare = pd.DataFrame(
        data=data_comp, columns=[str(i) for i in range(data_comp.shape[1])]
    ).T
    compare["diff(%)"] = np.abs(compare[1] - compare[0]) / np.abs(compare[0])
    compare["bound"] = bound_i

    return x_adv_num, compare

In [5]:
def find_bounds_box(clf, box):
    """
    Convert a decision box of list of c(i,j) to a dictionary of the 
    feature index (as keys) and the thresholds (as values).
    Args:
        clf: RandomForest classifier
        box: a list of `c(i,j)` that shows the decision box of the trees. 
            i denotes the tree number in the ensemble and j shows the leaf of 
            the tree.
    Returns:
        dict:
            keys are the feature index and values are the thresholds of the decisoin boundary
            associated to the feature (a list of [min, max]).
    """
    box_bounds = {}
    for box_i in box:
        tree_num = int(box_i[2:-1].split(",")[0])
        leaf_num = box_i[2:-1].split(",")[1]
        
        dict_boundaries = find_boundaries(clf.estimators_[tree_num])
        bi = dict_boundaries[leaf_num]
        
        del bi['value']
        
        for ki in bi.keys():
            if ki in box_bounds.keys():
                box_bounds[int(ki)][0] = max(box_bounds[ki][0], bi[ki][0])
                box_bounds[int(ki)][1] = min(box_bounds[ki][1], bi[ki][1])
            else:
                box_bounds[int(ki)] = bi[ki]
    box_bounds = {k:box_bounds[k] for k in sorted(list(box_bounds.keys()))}
    return box_bounds

def combine_boxes(clf, boxes):
    """
    Loops on all the boxes (as the list of c(i,j)) and finds the largest box around them. 
    This is used to check if the combination of the boxes is still growing or not. 
    
    Args:
        clf: RandomForest classifier
        boxes: list of lists of c(i,j) => list of decision boxes. 
    Returns:
        dict:
            a dictionary of keys: the feature index, values: the thresholds
            showing the decision box. 
    """
    final_box_bounds = {}
    for box in boxes:
        box_bounds = find_bounds_box(clf, box)
        
        for ki in box_bounds.keys():
            if ki in final_box_bounds.keys():
                final_box_bounds[ki][0] = min(final_box_bounds[ki][0], box_bounds[ki][0])
                final_box_bounds[ki][1] = max(final_box_bounds[ki][1], box_bounds[ki][1])
            else:
                final_box_bounds[ki] = box_bounds[ki]
    return final_box_bounds

In [6]:
def find_bound_mat(clf, bound_list, n_features):
    """
    Converts the dictionary of {feature index: threshold}
    to a numpy matrix of 2 x n_features.
    
    Args:
        clf: RandomForest classifier
        bound_list: list of c(i,j) showing the boundary
        n_features: number of features
        
    Returns:
        array-like: 2 x n_features first row is the min and the second 
        row is tha max of the features in the decision box. 
    """
    bound = find_bounds_box(clf, bound_list)
    bound_mat = np.zeros((2,n_features))
    for i in range(n_features):
        bound_i = bound.get(i, [-np.inf, np.inf])
        bound_mat[0, i] = bound_i[0]
        bound_mat[1, i] = bound_i[1]

    return bound_mat

In [7]:
def box_inside(box0, box1):
    """
    Checks if box1 is inside box0. Used for combining the boxes. 
    
    Args:
        box0: array-like 2 x n_features
        box1: array-like 2 x n_features
    
    Returns:
        True/False if box1 is insed box0
    """
    mini = box0[0,:]
    minj = box1[0,:]
    maxi = box0[1,:]
    maxj = box1[1,:]
    
    inds = (mini <= minj) & (maxi >= maxj)
    return np.all(inds)

def box_complementary(box0, box1):
    """
    Checks if box0 and box1 are only different in one
    feature and complement each other so they can be combined
    into a larger one. 
    
    Args:
        box0: array-like 2 x n_features
        box1: array-like 2 x n_features
        
    Returns:
        Bool:
            if they are complementary
        int:
            the index of feature to be combined
        float:
            the new min
        float:
            the new max
    """
    
    inequl_bounds = ((box0[0,:] != box1[0,:]) |
                     (box0[1,:] != box1[1,:]))

    inds = np.where(inequl_bounds)[0]

    box0[:,inds]
    
    if len(inds)==1:
        comp_bound = (box0[0,:] != box1[0,:]) & (box0[1,:] != box1[1,:])
        inds_comp = np.where(comp_bound)[0]

        if len(inds_comp) == 1:
            
            if box0[0,inds_comp]<=box1[1,inds_comp]:
                return True, inds_comp, box1[0,inds_comp], box0[1,inds_comp]
            if box1[0,inds_comp]<=box0[1,inds_comp]:
                return True, inds_comp, box0[0,inds_comp], box1[1,inds_comp]
            
    return False, None, None, None

In [8]:
def find_exact_box(clf, boxes, n_features):
    """
    Merges the boxes, by checking if they are indise each other or
    complementary. 
    
    Args:
        clf: RandomForest Classifier
        boxes: list of list of c(i,j) showing the boundary
        n_features: number of features
        
    Returns:
        2 x n_features showing the combined decision box
    """
    boxes_bound = []
    for box in boxes:
        boxes_bound.append(find_bound_mat(clf, box, n_features))
    
    def _combine_one_box(current_boxes, new_box):
        
        if len(current_boxes) == 0:
            return [new_box]
        
        box1 = new_box
        box1_added = False
        for ibox in range(len(current_boxes)):
            box0 = current_boxes[ibox]
            
            if box_inside(box0, box1):

                box0 = box1
                box1_added = True

            elif box_inside(box1, box0):
                box1_added = True

            else:
                complemetary, inds_comp, mini, maxi = box_complementary(box0, box1)
                if complemetary:
                    box0[0,inds_comp[0]] = mini
                    box0[1,inds_comp[0]] = maxi
                    box1_added = True

        if not box1_added:
            current_boxes.append(box1)
            
        return current_boxes

    current_boxes = []
    for ind in range(len(boxes_bound)):
        current_boxes = _combine_one_box(current_boxes, boxes_bound[ind])
    
    new_len = len(current_boxes)
    old_len = len(boxes_bound)
    
    while new_len < old_len and old_len > 1:
        current_boxes_ = []
        for ind in range(len(current_boxes)):
            current_boxes_ = _combine_one_box(current_boxes_, current_boxes[ind])
        
        new_len = len(current_boxes_)
        old_len = len(current_boxes)        
        
        current_boxes = current_boxes_
        
    return current_boxes
    

In [9]:
# Find all c's with only one change without changing the outcome

def find_lower_preds(ind, prediction, c_list, adv_weights, c_weights, all_c):
    """
    Creates new decision boundaries to reduce the needed SAT runs, by changing each of the
    c(i,j)'s with one in the l_inf ball and lower/higer value depending on the prediciotn being 0/1. 
    
    Args:
        ind: which c(ind,.) to change
        prediction: true prediction that we don't want to change
        c_list: the decision box we want to create the new boxes from 
        adv_weights: a dictionary that provides the value of the boxes
        c_weights: conttains all the weights
        all_c: all the c(i,j)'s in l_inf ball. 
    
    Returns:
        list:
            list of new decision boxes with only one c(ind,j) changed
    """
    new_lists = []
    si = f"c({ind},"
    clist = [ki for ki,_ in adv_weights.items()]
    curr_c = [ki for ki, vi in adv_weights.items() if ki.startswith(si)][0]

    if prediction:
        val_i = [vi for ki, vi in adv_weights.items() if ki.startswith(si)][0]
        ci_higher = [ci for ci in all_c if c_weights[ci] >= val_i and ci.startswith(si) and ci!=curr_c]
        if len(ci_higher)>0:
            for cli in ci_higher:
                new_l = set(clist) - set([curr_c])
                new_l.add(cli)
                
                new_l = sorted(list(new_l))
                
                if new_l not in c_list:
                    new_lists.append(new_l)

    else:
        val_i = [vi for ki, vi in adv_weights.items() if ki.startswith(si)][0]
        ci_lower = [ci for ci in all_c if c_weights[ci] <= val_i and ci.startswith(si) and ci!=curr_c]
        if len(ci_lower)>0:
            for cli in ci_lower:
                new_l = set(clist) - set([curr_c])
                new_l.add(cli)
                
                new_l = sorted(list(new_l))

                if new_l not in c_list:
                    new_lists.append(new_l)
                
    return new_lists

In [10]:
# To find combinations of two c's to change without changing the outcome

def get_adv_weight_ind(adv_weights, ind):
    """
    Get the value of the C(ind,.) from `adv_weights`. 
    
    Args:
        adv_weights: dict with keys being c(i,j) and values being the decision values 
        ind: the tree num to get its value in the current decision box
    
    Return:
        float: 
            the value of the decision box of ind'th tree.
    """
    si = f"c({ind},"
    cur_val = [vi for ci,vi in adv_weights.items() if ci.startswith(si)][0]
    return cur_val


def get_name_weight(ind_1, ind_2, c_weights, all_c):
    """
    Gets all the values and names associated with ind_1 and ind_2'th trees. 
    
    Args:
        ind_1: int, first tree
        ind_2: int, second tree
        c_weights: dictionary of all weights in tree leaves
        all_c: the c(i,j)'s in l_inf ball
    
    Returns:
        4 values, names and weights
    """
    names_1 = [ci for ci in all_c if f"c({ind_1}," in ci]
    weights_1 = [c_weights[ci] for ci in all_c if f"c({ind_1}," in ci]

    names_2 = [ci for ci in all_c if f"c({ind_2}," in ci]
    weights_2 = [c_weights[ci] for ci in all_c if f"c({ind_2}," in ci]

    return names_1, weights_1, names_2, weights_2

def get_replacements(ind_1, ind_2, prediction, c_weights, adv_weights, all_c):
    """
    Find the two-set combinations of c(ind_1,.) and c(ind_2,.) to replace the original ones
    without changing the predictions. 
    """
    cur_val = get_adv_weight_ind(adv_weights, ind_1) + get_adv_weight_ind(adv_weights, ind_2)
    names_1, weights_1, names_2, weights_2 = get_name_weight(ind_1, ind_2, c_weights, all_c)
    if prediction:
        findings = np.where(np.tile(weights_1, (len(weights_2),1)) + np.tile(weights_2, (len(weights_1),1)).T >= cur_val)
    else:
        findings = np.where(np.tile(weights_1, (len(weights_2),1)) + np.tile(weights_2, (len(weights_1),1)).T <= cur_val)
    
    
    return [(names_1[ci[1]], names_2[ci[0]]) for ci in zip(findings[0], findings[1])]


def replace_names(ind_1, ind_2, adv_weights, replacements):
    """
    Creates new lists of c(i,j)'s by replacing ind_1 and ind_2 with the ones in the 
    l_inf ball without changing the predictions (weights are less than or equal to the 
    original ones). 
    """
    si = f"c({ind_1},"
    cur_name_1 = [ci for ci,vi in adv_weights.items() if ci.startswith(si)][0]
    si = f"c({ind_2},"
    cur_name_2 = [ci for ci,vi in adv_weights.items() if ci.startswith(si)][0]
    
    clist = [ki for ki,_ in adv_weights.items()]
    
    all_new_c = []
    for repl_i in replacements:
        new_l = set(clist) - set([cur_name_1]) -set([cur_name_2])
        new_l.add(repl_i[0])
        new_l.add(repl_i[1])
        all_new_c.append(sorted(list(new_l)))
    
    return all_new_c

# model and data

In [12]:
data = load_breast_cancer()
feature_names = data["feature_names"]
X = data['data']
y = data["target"] == 1
feature_names = data["feature_names"]
ntrees = 50
clf = RandomForestClassifier(n_estimators=ntrees, max_depth=5).fit(X, y)

In [13]:
all_thresh = get_ens_thresh(clf)
data_ = disc_ens_data(X, all_thresh)
print(data_.shape)
data_.head()

(569, 671)


,21_28.864999771118164,2_78.73500061035156,22_117.54999923706055,22_117.44999694824219,13_32.6299991607666,13_32.78000068664551,9_0.06602999940514565,9_0.05591000057756901,9_0.0578600000590086,9_0.07868999987840652,...,27_0.11134999990463257,27_0.13544999808073044,27_0.15839999914169312,3_306.0500030517578,3_649.6000061035156,22_114.69999694824219,22_114.6500015258789,13_39.35499954223633,3_687.2000122070312,22_110.25
0,False,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
1,False,True,True,True,True,True,False,True,False,False,...,True,True,True,True,True,True,True,True,True,True
2,False,True,True,True,True,True,False,True,True,False,...,True,True,True,True,True,True,True,True,True,True
3,False,False,False,False,False,False,True,True,True,True,...,True,True,True,True,False,False,False,False,False,False
4,False,True,True,True,True,True,False,True,True,False,...,True,True,True,True,True,True,True,True,True,True


In [14]:
var_x = create_var_x(all_thresh)

In [15]:
nbits = 8
new_nbits = int(np.ceil(np.log2(ntrees)) + nbits)

In [55]:
epsilon = 0.1


index = 111
sample = X[index:index+1, :]

# upper bound

In [56]:
core_constraints, soft_constraints, c_weights, all_c = create_all_smt(
    clf, var_x, sample, epsilon
)
list_val_, list_c_ = list_c_val(c_weights, nbits)
sum_const, seq_num = sum_loop(list_val_, list_c_, new_nbits)
const_class = const_larger(nbits, ntrees, seq_num)

True


In [57]:
print("number of leaves: ", seq_num)

number of leaves:  801


In [58]:
s = Solver()

s.set("timeout", 20000)
for ci in core_constraints:
    s.add(ci)
    
for ci in sum_const:
    s.add(ci)
    
for ci in const_class:
    s.add(ci)

s.add([Not(Bool("class")) if clf.predict(sample)[0] else Bool("class")])


beg_time = time.time()

if s.check() == sat:
    adv_weights = get_output(s, c_weights)
    print(np.mean([v for k,v in adv_weights.items()]), clf.predict_proba(sample))
else: 
    print("unsat")
    
end_time = time.time()
print()
print("+++++++++")
print("sat check time + prepare output: ", end_time-beg_time)

unsat

+++++++++
sat check time + prepare output:  0.720613956451416


# lower bound

In [59]:
core_constraints, soft_constraints, c_weights, all_c = create_all_smt(
    clf, var_x, sample, epsilon, lower_bound=True
)
list_val_, list_c_ = list_c_val(c_weights, nbits)
sum_const, seq_num = sum_loop(list_val_, list_c_, new_nbits)
const_class = const_larger(nbits, ntrees, seq_num)

True


In [60]:
s = Solver()

s.set("timeout", 5000)
for ci in core_constraints:
    s.add(ci)
    
for ci in sum_const:
    s.add(ci)
    
for ci in const_class:
    s.add(ci)

s.add([Bool("class") if clf.predict(sample)[0] else Not(Bool("class"))])

In [61]:
s.check()

sat

In [62]:
adv_weights = get_output(s, c_weights)
np.mean([v for k,v in adv_weights.items()])

0.853798218051938

In [63]:
clf.predict_proba(sample)

array([[0.08344574, 0.91655426]])

In [64]:
adv_weights = get_output(s, c_weights)
c_list = []
c_list.append(sorted(list(adv_weights.keys())))
run = True
count = 0
stop_count = 0
new_box = {}
added_varx = {}
stop_division = 5

while run:
    
    count += 1
    
    # do we need to remove the var_x's too? maybe not since this is the same as
    # removing c(i,j) list. But could accelerate the search?
    # This reduces the search space a lot and removes some of the possible
    # regions
    s.add(Not(And([v if s.model()[v] else Not(v) for k,v in var_x.items()])))
    
    # removes the currently found box from the search 
    s.add(Not(And([Bool(k) for k,v in adv_weights.items()])))
    
    new_list = []
    
    # adds all the new combinations that we know won't change the results
    # based on the current result. This includes all the one-var replacements 
    # and two-var replacements
    for ind in range(len(adv_weights)):
        new_c = find_lower_preds(ind, clf.predict(sample)[0], c_list, adv_weights, c_weights, all_c)
        if len(new_c)>0:
            for ci in new_c:
                if sorted(ci) not in c_list and ci not in new_list:
                    s.add(Not(And([Bool(k) for k in ci])))
                    new_list.append(sorted(ci))
        
        for ind_2 in range(ind+1, len(adv_weights)):
            replacements_2d = get_replacements(ind, ind_2, clf.predict(sample)[0], c_weights, adv_weights, all_c)
            if len(replacements_2d) > 0:
                new_c = replace_names(ind, ind_2, adv_weights, replacements_2d)
                for ci in new_c:
                    if sorted(ci) not in c_list and ci not in new_list:
                        s.add(Not(And([Bool(k) for k in ci])))
                        new_list.append(sorted(ci))
    
    # check if the added constraints are still statisfied
    # if they are add the new list to the outcome list
    if s.check() == sat:
        for ci in new_list:
            c_list.append(ci)
        adv_weights = get_output(s, c_weights)
        cur_c = list(adv_weights.keys())
        if sorted(cur_c) not in c_list:
            c_list.append(sorted(cur_c))
            print(count, np.mean([v for k,v in adv_weights.items()]))
        
        # an early stopping heuristic: if the 
        # area of added bounds does not grow
        # start the stopping counts. 
        if count%stop_division == 0:
            old_box = copy.deepcopy(new_box)
            
            # Todo: do not do this everytime from scratch, add the new regions only
            new_box = combine_boxes(clf, c_list)
            if new_box==old_box:
                stop_count += 1
                stop_division -= 1
                print("stop_count: ", stop_count)
                if stop_count >= 5:
                    run = False
            else:
                stop_count = 0
                stop_division = 5
    
    else:
        print("unsat")
        run = False

1 0.8689886942424141
2 0.8688473514862304
3 0.8491144804059361
4 0.8489731376497524
5 0.8875601228138428
6 0.8725601228138428
7 0.8526859089773646
8 0.8339731376497523
9 0.9075601228138427
10 0.8925601228138427
stop_count:  1
11 0.8726859089773646
12 0.8875445662211809
stop_count:  2
13 0.892418780057659
14 0.869114480405936
15 0.8857601228138426
stop_count:  3
16 0.8707601228138426
stop_count:  4
17 0.8658859089773648
stop_count:  5


In [65]:
len(c_list)

7279

In [66]:
print( "Number of decision boxes: ", len(c_weights) )

Number of decision boxes:  802


In [83]:
for ci in c_list:
    if np.mean([c_weights[k] for k in ci])<.5:
        print(np.mean([c_weights[k] for k in ci]))

In [68]:
exact_bounds_mat = find_exact_box(clf, c_list, sample.shape[1])
len(exact_bounds_mat)

46

In [69]:
# convert matrix format to dictionary so can be used in generating adversarial example. 

exact_bounds = []

for eb in range(len(exact_bounds_mat)):
    dict_true = {f"x({i}_{v})":set([True]) for i, v in enumerate(exact_bounds_mat[eb][0]) if not np.isinf(v)}
    dict_false = {f"x({i}_{v})":set([False]) for i, v in enumerate(exact_bounds_mat[eb][1]) if not np.isinf(v)}
    dict_true.update(dict_false)
    exact_bounds.append(dict_true)
len(exact_bounds)

46

In [70]:
all_inds = []
for i in range(1000):
    ind = np.random.randint(0, len(exact_bounds))
    all_inds.append(ind)
    exact_box = exact_bounds[ind]
    example, compare = create_adv_sample(exact_box, sample)
    if i%100 == 0:
        print(i, ind)
    if clf.predict(example) != clf.predict(sample):
        print(clf.predict_proba(example))
        break

0 17
100 13
200 25
300 3
400 7
500 26
600 5
700 34
800 30
900 11


In [71]:
# example = find_samples(exact_box, sample)
clf.predict_proba(example), clf.predict_proba(sample)

(array([[0.25153918, 0.74846082]]), array([[0.08344574, 0.91655426]]))

In [72]:
c_example = [f"c({tree},{leaf})" for tree, leaf in enumerate(clf.apply(example)[0])]
c_example in c_list

False

In [73]:
np.mean([c_weights[ci] for ci in c_example])

0.7484608228568516

In [74]:
example, compare = create_adv_sample(exact_box, sample)

In [75]:
compare.sort_values(by=["diff(%)"], ascending=False)

,0,1,diff(%),bound
16,0.051010,-9085.348857,178110.171870,"[-inf, inf]"
8,0.173500,-22.098317,128.367821,"[-inf, 0.26365000009536743]"
2,82.150000,-9192.257020,112.896008,"[-inf, 98.56999969482422]"
23,527.400000,-57893.611501,110.771732,"[-inf, 711.3000183105469]"
20,13.330000,-1424.235262,107.844356,"[-inf, 16.22000026702881]"
13,20.480000,-2051.967667,101.193734,"[-inf, 33.0049991607666]"
22,89.000000,-8714.207316,98.912442,"[-inf, 122.05000305175781]"
25,0.225000,-20.256745,91.029976,"[-inf, 0.43675000965595245]"
12,2.711000,-194.766569,72.843072,"[-inf, 4.09850001335144]"
4,0.099330,-6.610781,67.553725,"[-inf, 0.11549999937415123]"


In [76]:
exact_box = exact_bounds[5]
example, compare = create_adv_sample_x(exact_box, sample)

In [77]:
compare

,0,1,diff(%),bound
0,12.630000,12.630000,0.0,"[10.312999725341797, inf]"
1,20.760000,20.760000,0.0,"[16.835000038146973, inf]"
2,82.150000,82.150000,0.0,"[-inf, 98.56999969482422]"
3,480.400000,480.400000,0.0,"[-inf, 778.5]"
4,0.099330,0.099330,0.0,"[-inf, 0.11549999937415123]"
5,0.120900,0.120900,0.0,"[-inf, 0.15139999985694885]"
6,0.106500,0.106500,0.0,"[0.033740000799298286, inf]"
7,0.060210,0.060210,0.0,"[0.05591999925673008, inf]"
8,0.173500,0.173500,0.0,"[-inf, 0.26365000009536743]"
9,0.070700,0.070700,0.0,"[0.053300000727176666, inf]"


In [78]:
pd.concat([pd.DataFrame(exact_bounds[3]), pd.DataFrame(exact_bounds[4])]).reset_index(drop=True).T

,0,1
x(0_11.170000076293945),{True},{True}
x(1_16.835000038146973),{True},{True}
x(6_0.033740000799298286),{True},{True}
x(9_0.053300000727176666),{True},{True}
x(14_0.002832000027410686),{True},{True}
x(17_0.01871499978005886),{True},{True}
x(18_0.010335000231862068),{True},{True}
x(19_0.0013150000013411045),{True},{True}
x(27_0.11574999988079071),{False},{False}
x(28_0.15654999762773514),{True},NaN


In [79]:
exact_bounds_mat[0][0] == exact_bounds_mat[1][0]

array([False,  True,  True,  True,  True,  True,  True, False,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True, False,  True,  True,  True,  True,  True,
        True, False, False])

In [80]:
exact_bounds_mat[0][1] == exact_bounds_mat[1][1]

array([ True,  True,  True,  True,  True,  True,  True, False,  True,
        True, False,  True,  True,  True,  True,  True,  True,  True,
        True,  True, False, False,  True, False,  True,  True,  True,
        True,  True,  True])

In [81]:
exact_bounds_mat[0][:,4], exact_bounds_mat[1][:,4]

(array([  -inf, 0.1155]), array([  -inf, 0.1155]))

In [82]:
exact_bounds_mat[0][:,5], exact_bounds_mat[1][:,5]

(array([  -inf, 0.1514]), array([  -inf, 0.1514]))